# Concatenar y apendizar data sets

In [1]:
import pandas as pd

In [10]:
red_wine = pd.read_csv("../datasets/wine/winequality-red.csv", sep = ";")
white_wine = pd.read_csv("../datasets/wine/winequality-white.csv", sep=";")

In [16]:
red_wine.columns.values

array(['fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol',
       'quality'], dtype=object)

In [18]:
white_wine.columns.values

array(['fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol',
       'quality'], dtype=object)

En pyhton, tenemos dos tipos de ejes:
- axis = 0 denota el eje horizontal
- axis = 1 denota el eje vertical

In [25]:
wine_data = pd.concat([red_wine, white_wine], axis = 0)

In [27]:
wine_data.shape

(6497, 12)

## Datos distribuidos

In [8]:
import pandas as pd
data = pd.read_csv("../datasets/distributed-data/001.csv")
data.shape

(1461, 4)

* Importar el primer fichero
* Hacemos un bucle para ir recorriendo todos y cada uno de los ficheros
    * Importante tener una consistencia en el nombre de los ficheros
    * Importamos los ficheros uno a uno
    * Cada uno de ellos debe añadirse al final del primer fichero que ya habíamos cargado
* Repetimos el bucle hasta que no queden ficheros
    

In [17]:
filepath = "../datasets/distributed-data/"

data = pd.read_csv("../datasets/distributed-data/001.csv")

final_length = len(data)

for i in range(2,333):
    if i < 10:
        filename = "00" + str(i)
    elif i < 100:
        filename = "0" + str(i)
    else:
        filename = str(i)
    
    file = filepath + filename + ".csv"
    
    temp_data = pd.read_csv(file)
    final_length += len(temp_data)
    
    data = pd.concat([data, temp_data], axis=0)
    
print("Exito de carga")

Exito de carga


In [20]:
data.shape[0] == final_length

True

## Joins de datasets

In [21]:
filepath = "../datasets/athletes/"

In [2]:
# DF principal
data_main = pd.read_csv("../datasets/athletes/Medals.csv", encoding ="ISO-8859-1")
data_athlets = data_main["Athlete"].unique().tolist()

# DF con información de pais
data_country = pd.read_csv("../datasets/athletes/Athelete_Country_Map.csv", encoding = "ISO-8859-1")

# DF con información de deportes
data_sports = pd.read_csv("../datasets/athletes/Athelete_Sports_Map.csv", encoding = "ISO-8859-1")

In [7]:
# Quitar duplicados al dataframe Country, debido a que un unico atleta puede tener dos paises o más
data_country_dp = data_country.drop_duplicates(subset = "Athlete")
len(data_country_dp) == len(data_athlets)

True

In [8]:
data_main_country = pd.merge(left = data_main, right=data_country_dp, 
                             left_on="Athlete", right_on = "Athlete")
data_main_country.shape

(8618, 9)

In [9]:
# Quitar duplicados al dataframe Sports, debido a que un unico atleta puede tener dos deportes o más
data_sports_dp = data_sports.drop_duplicates(subset = "Athlete")
len(data_country_dp) == len(data_athlets)

True

In [10]:
data_final = pd.merge(left= data_main_country, right = data_sports_dp, 
                      on="Athlete")
data_final.shape

(8618, 10)

## Tipos de Joins

In [5]:
from IPython.display import Image
import numpy as np

In [11]:
out_athletes = np.random.choice(data_main["Athlete"], size = 6, replace = False)

In [20]:
data_country_dlt = data_country_dp[(~data_country_dp["Athlete"].isin(out_athletes)) & 
                                   (data_country_dp["Athlete"] != "Michael Phelps")]

data_sports_dlt = data_sports_dp[(~data_sports_dp["Athlete"].isin(out_athletes)) & 
                                   (data_sports_dp["Athlete"] != "Michael Phelps")]

data_main_dlt = data_main[(~data_main["Athlete"].isin(out_athletes)) & 
                                   (data_main["Athlete"] != "Michael Phelps")]

In [28]:
print("country_dlt = " + str(len(data_country_dlt))
     + " sports_dlt = " + str(len(data_sports_dlt))
     + " main_dlt = " + str(len(data_main_dlt)))

country_dlt = 6949 sports_dlt = 6949 main_dlt = 8607


### Inner Join
* Devuelve un data frame con las filas que tienen valor tanto en el primer como en el segundo data frame que estamos uniendo
* El número de filas será igual al número de filas **comunes** que tengan ambos data sets
    * Data Set A tiene 60 filas
    * Data Set B tiene 50 filas
    * Ambos comparten 30 filas
    * Entonces A Inner Join B tendrá 30 filas
* En términos de teoría de conjuntos, se trata de la intersección de ls dos conjuntos

In [34]:
merged_inner = pd.merge(left = data_main, right = data_country_dlt,
                       how = "inner", left_on = "Athlete", right_on = "Athlete")
merged_inner.head()

,Athlete,Age,Year,Closing Ceremony Date,Gold Medals,Silver Medals,Bronze Medals,Total Medals,Country
0,Natalie Coughlin,25.0,2008,08/24/2008,1,2,3,6,United States
1,Natalie Coughlin,21.0,2004,08/29/2004,2,2,1,5,United States
2,Natalie Coughlin,29.0,2012,08/12/2012,0,0,1,1,United States
3,Aleksey Nemov,24.0,2000,10/01/2000,2,1,3,6,Russia
4,Alicia Coutts,24.0,2012,08/12/2012,1,3,1,5,Australia


### Left Join
* Devuelve un data frame con las filas que tuvieran valor en el dataset de la izquierda, sin importar si tienen correspondencia en el de la derecha o no-
* Las filas de data frame final que no correspondan a ninguna fila del data frame derecho, tendrán NAs en las columnas del data frame derecho.
* El número de filas será igual al número de filas del data frame izquierdo
    * Data Set A tiene 60 filas
    * Data Set B tiene 50 filas
    * Entonces A Left Join B tendrá 60 filas
* En términos de teoría de conjuntos, se trata del propio data set de la izquierda quien, además tiene la intersección en su interior.

In [53]:
merged_left = pd.merge(left = data_main, right = data_country_dlt,
                      how = "left", on="Athlete")

merged_left[merged_left["Country"].isnull()]

,Athlete,Age,Year,Closing Ceremony Date,Gold Medals,Silver Medals,Bronze Medals,Total Medals,Country
0,Michael Phelps,23.0,2008,08/24/2008,8,0,0,8,NaN
1,Michael Phelps,19.0,2004,08/29/2004,6,0,2,8,NaN
2,Michael Phelps,27.0,2012,08/12/2012,4,2,0,6,NaN
4757,Christian Schulte,28.0,2004,08/29/2004,0,0,1,1,NaN
5158,Trine Haltvik,35.0,2000,10/01/2000,0,0,1,1,NaN
5300,Brett McClure,23.0,2004,08/29/2004,0,1,0,1,NaN
5770,Yevgeny Plyushchenko,27.0,2010,02/28/2010,0,1,0,1,NaN
5792,Yevgeny Plyushchenko,23.0,2006,02/26/2006,1,0,0,1,NaN
5810,Yevgeny Plyushchenko,19.0,2002,02/24/2002,0,1,0,1,NaN
7497,Natalie Cook,25.0,2000,10/01/2000,1,0,0,1,NaN


### Right Join
* Devuelve un dataframe con las filas que tuvieran valor en el dataset de la derecha, sin importar si tienen correspondencia en el de la izquierda o no.
* Las filas del data frame final que no correspondan a ninguna fila del data frame izquierdo, tendrán NAs en las columnas del data frame izquierdo.
* El número de filas será igual al número de filas del data frame derecho.
    * Data Set A tiene 60 filas
    * Data Set B tiene 50 filas
    * Entonces A Right join B tendrá 50 filas
* En términos de teoría de conjuntos, se trata del propio data set de la derecha quien, además tiene la intersección en su interior.

In [51]:
merged_right = pd.merge(left = data_main_dlt, right = data_country_dp,
                      how = "right", on="Athlete")
len(merged_right)

8614

### Outer Join
* Devuelve un data frame con todas las filas de ambos, reemplazando las ausencias d euno o de otro con NAs en la región específica.
* Las filas del data frame final que no correspondan a ninguna fila del data frame derecho (o izquierdo), tendrán NAs en las columnas del data frame derecho (o izquerdo).
* El número de filas será igual al máximo número de filas de ambos data frames
    * Data Set A tiene 60 filas
    * Data Set B tienen 50 filas
    * Ambos comparten 30 filas
    * Entonces A Outer Join B tendrá 60 + 50 - 30 = 80 filas
* En términos de teoría de conjuntos, se trata de la unión de conjuntos.

In [56]:
data_country_jb = data_country_dlt.append(
    {
        "Athlete" : "José La Rosa",
        "Country" : "Perú"
    }, ignore_index = True
)

In [58]:
merged_outer = pd.merge(left = data_main, right = data_country_jb,
                       how = "outer", on = "Athlete")
len(merged_outer)

8619